In [1]:
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
import numpy as np 
import torch
import dgl
from dgl.data import AsNodePredDataset
from ogb.nodeproppred import DglNodePropPredDataset

In [11]:
graph = np.fromfile("/home/bear/workspace/single-gnn/data/raid/ogbn_products/graph.bin")

In [12]:
src = graph[::2]
dst = graph[1::2]

In [13]:
trainIds = np.fromfile("/home/bear/workspace/single-gnn/data/raid/ogbn_products/trainMask.bin",dtype=np.int8)

In [16]:
ids = torch.nonzero(torch.tensor(trainIds))
ids = ids.reshape(-1).to(np.int32)

tensor([     0,      1,      2,  ..., 196612, 196613, 196614])

In [18]:
nodeTable = torch.zeros(2449029,dtype=torch.int32)
nodeTable

tensor([0, 0, 0,  ..., 0, 0, 0], dtype=torch.int32)

In [20]:
nodeTable[ids] = 1
nodeTable

tensor([1, 1, 1,  ..., 0, 0, 0], dtype=torch.int32)

In [ ]:
dgl.fastFindNeighbor(nodeTable,src,dst)

In [24]:
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix
import numpy as np 
import torch
import dgl
import time 
import copy
## bfs 遍历获取基础子图
GRAPHPATH = "/home/bear/workspace/single-gnn/data/raid/uk-2006-05"
graph = np.fromfile(GRAPHPATH + "/graph.bin",dtype=np.int32)
trainIds = np.fromfile(GRAPHPATH + "/train_mask.bin",dtype=np.int8)
src = graph[::2]
dst = graph[1::2]
maxID = 77741046
ids = torch.nonzero(torch.tensor(trainIds))
ids = ids.reshape(-1).to(torch.int64)
# ids = torch.arange(int(maxID*0.01),dtype=torch.int64)
nodeTable = torch.zeros(maxID,dtype=torch.int32)
nodeTable[ids] = 1

raw_nodeTable = copy.deepcopy(nodeTable)
next_nodeTable = copy.deepcopy(nodeTable)

batch_size = 2
nodeTable = nodeTable.cuda()
src = torch.tensor(src)
dst = torch.tensor(dst)
src_batches = torch.chunk(src, batch_size, dim=0)
dst_batches = torch.chunk(dst, batch_size, dim=0)

In [29]:
src.shape

torch.Size([2965197340])

: 

In [28]:
batch = [src_batches, dst_batches]
for src_batch,dst_batch in zip(*batch):
    print(src_batch.shape)
    print(dst_batch.shape)
    print("-"*15)

torch.Size([1482598670])
torch.Size([1482598670])
---------------
torch.Size([1482598670])
torch.Size([1482598670])
---------------


In [ ]:
# src = torch.tensor(src).cuda()
# dst = torch.tensor(dst).cuda()


repeats = 3
start = time.time()
for index in range(1,repeats+1):
    for src_batch,dst_batch in zip(*batch):
        tmp_nodeTabel = copy.deepcopy(raw_nodeTable)
        print(f"before {index} BFS has {nodeTable.sum()} nodes, {nodeTable.sum() * 1.0 / maxID * 100 :.2f}% of total nodes")
        dgl.fastFindNeighbor(tmp_nodeTabel, src_batch, dst_batch)
        print(f"after {index} BFS has {nodeTable.sum()} nodes, {nodeTable.sum() * 1.0 / maxID * 100 :.2f}% of total nodes")
        print('-'*10)
        next_nodeTable = next_nodeTable | tmp_nodeTabel
print(f"all bfs cost {time.time()-start:.3f}s")
## loading graph